# loading libraries

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
import glob
import pathlib
import pandas as pd
import numpy as np
import darts
from darts import TimeSeries
import matplotlib.pyplot as plt
from statsmodels.tsa.stattools import adfuller
import os
import glob
from tqdm import tqdm
from darts.dataprocessing.transformers.scaler import Scaler

# Helping Functions


In [ ]:
import numpy as np


def calculate_metrics(actual, predicted):
    # Convert inputs to numpy arrays for easier calculations
    actual = np.array(actual)
    predicted = np.array(predicted)

    # Calculate individual metrics
    mae = np.mean(np.abs(predicted - actual))
    rmse = np.sqrt(np.mean((predicted - actual) ** 2))
    mape = np.mean(np.abs((predicted - actual) / actual)) * 100
    mse = np.mean((predicted - actual) ** 2)

    metrics = {"MAE": mae, "RMSE": rmse, "MAPE": mape, "MSE": mse}

    return metrics

# Data Reading 😶

In [ ]:
fileName = "amazon"
df = pd.read_csv(f"../ProcessedData/{fileName}.csv")
df.head()

In [ ]:
df = df[["Date", "Close"]]
df = df.drop_duplicates()
df.head()

# stationary Tesst

In [ ]:
def adfuller_test(values):
    result = adfuller(values)
    labels = [
        "ADF Test Statistic",
        "p-value",
        "#Lags Used",
        "Number of Observations Used",
    ]
    for value, label in zip(result, labels):
        print(label + " : " + str(value))
    if result[1] <= 0.05:
        print(
            "P value is less than 0.05 that means we can reject the null hypothesis(Ho). Therefore we can conclude that data has no unit root and is stationary"
        )
    else:
        print(
            "Weak evidence against null hypothesis that means time series has a unit root which indicates that it is non-stationary "
        )

In [ ]:
adfuller_test(df["Close"])

In [ ]:
adfuller_test(df["Close"].diff(1).iloc[1:])

## Spliting Data into Training & Testing Data

In [ ]:
from darts import TimeSeries
import numpy as np
import matplotlib.pyplot as plt

# Assuming df is your DataFrame containing daily data
series = TimeSeries.from_dataframe(
    df, "Date", "Close", freq="1D", fill_missing_dates=True, fillna_value=0
)


split_point = 0.80

train_series, test_series = series.split_after(split_point)

# Set the figure size and style
plt.figure(figsize=(18, 6))
# Plot the training and testing data
train_series.plot(label="Training Data", color="blue", linewidth=1.5, marker="o")
test_series.plot(label="Testing Data", color="orange", linewidth=1.5, marker="o")

# Add title and labels
plt.title("Training and Testing Data")
plt.xlabel("Date")
plt.ylabel("Amazon Close Price")

# Add grid lines
plt.grid(True)

# Add legend
plt.legend()

# Display the plot
plt.show()

## Exponential Smoothing Model

In [ ]:
from darts.models.forecasting.exponential_smoothing import ExponentialSmoothing
from darts.utils.utils import ModelMode
from darts.utils.utils import SeasonalityMode

expo_smoot = ExponentialSmoothing(
    trend=ModelMode.ADDITIVE,
    damped=True,
    seasonal=SeasonalityMode.ADDITIVE,
    seasonal_periods=4,
    random_state=1999,
    remove_bias=True,
    smoothing_seasonal=0.3,
)

expo_smoot.fit(train_series)

# Print the summary of the trained model
print(expo_smoot.model.summary())

### GridSearch for Exponential Smoothing

In [ ]:
# from darts.models.forecasting.exponential_smoothing import ExponentialSmoothing
# from darts.utils.utils import ModelMode, SeasonalityMode
# from darts.metrics import mae
# import numpy as np

# # Define a range of hyperparameters for the grid search
# trend_modes = [ModelMode.ADDITIVE, ]
# damped_values = [True, False]
# seasonality_modes = [SeasonalityMode.ADDITIVE]
# seasonal_periods_values = [4,7,10, 14,17,21, 30]  # You can add more values
# smoothing_seasonal_values = [0.1, 0.3, 0.5]  # You can add more values
# remove_biass =  [True,False]
# smoothing_trends = [ 0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9]
# smoothing_levels = [ 0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9]
# best_mae = np.inf
# best_params = None

# # Perform grid search
# for trend_mode in trend_modes:
#     for damped in damped_values:
#         for seasonality_mode in seasonality_modes:
#             for seasonal_periods in seasonal_periods_values:
#                 for smoothing_seasonal in smoothing_seasonal_values:
#                     for remove_bias in remove_biass:
#                         for smoothing_trend in smoothing_trends:
#                             for smoothing_level in smoothing_levels:

#                                 expo_smoot = ExponentialSmoothing(
#                                     trend=trend_mode,
#                                     damped=damped,
#                                     seasonal=seasonality_mode,
#                                     seasonal_periods=seasonal_periods,
#                                     random_state=1999,
#                                     remove_bias=remove_bias,
#                                     smoothing_seasonal=smoothing_seasonal,
#                                     smoothing_trend = smoothing_trend,
#                                     smoothing_level = smoothing_level
#                                 )

#                                 expo_smoot.fit(train_series)
#                                 forecast = expo_smoot.predict(len(test_series))
#                                 mae_score = mae(test_series, forecast)

#                                 if mae_score < best_mae:
#                                     best_mae = mae_score
#                                     best_params = {
#                                         'trend': trend_mode,
#                                         'damped': damped,
#                                         'seasonality': seasonality_mode,
#                                         'seasonal_periods': seasonal_periods,
#                                         'smoothing_seasonal': smoothing_seasonal
#                                     }

# print("Best parameters:", best_params)
# print("Best MAE:", best_mae)

In [ ]:
def make_plots(
    input_window, ground_truth, forecast, bypass_information, result_plot_path
):
    plt.figure(figsize=(30, 6))
    input_window.plot(label="Input Data", marker="o")
    forecast.plot(label="Predicted", marker="o")
    ground_truth.plot(label="Ground Truth", marker="o")

    combined_time_index = input_window.time_index.append(forecast.time_index).append(
        ground_truth.time_index
    )
    starting_date_of_input_data = input_window.time_index[0].strftime("%Y-%m-%d")
    ending_date_of_input_data = input_window.time_index[-1].strftime("%Y-%m-%d")
    starting_date_predicted = forecast.time_index[0].strftime("%Y-%m-%d")
    ending_date_of_predicted = forecast.time_index[-1].strftime("%Y-%m-%d")

    plt.xticks(
        combined_time_index, combined_time_index.strftime("%Y-%m-%d"), rotation=90
    )
    plt.title(
        f"Results of Input Data from {starting_date_of_input_data} to {ending_date_of_input_data} & Evaluation on from {starting_date_predicted} to {ending_date_of_predicted}",
        fontsize=16,
    )
    plt.ylabel("Amazon Close Price", fontsize=14)
    plt.xlabel("Dates", fontsize=14)
    plt.legend()

    plot_filename = f"{result_plot_path}/{bypass_information['window_size']}_{bypass_information['horizon']}_{bypass_information['slide_step']}.png"
    plt.savefig(plot_filename)
    # plt.show()
    plt.close()


# Function to predict and evaluate
def get_ground_truth(window_size, prediction_horizon, slide_step, test_series):
    ground_truth_list = []
    input_window_list = []
    num_predictions = len(test_series) - window_size - prediction_horizon + 1
    for i in range(0, num_predictions, slide_step):
        input_window = test_series[i : i + window_size]
        ground_truth = test_series[
            i + window_size : i + window_size + prediction_horizon
        ]
        ground_truth_list.append(ground_truth)
        input_window_list.append(input_window)

    return ground_truth_list, input_window_list

In [13]:
model_name = "ExponentionalSmoothing"
FileName = fileName
result_path = f"../ProcessedData/Results/{model_name}/{FileName}"
result_plot_path = (
    f"../ProcessedData/Results/{model_name}/{FileName}/{model_name}_Plots"
)
os.makedirs(result_path, exist_ok=True)
os.makedirs(result_plot_path, exist_ok=True)

window_sizes = [30, 45, 90]
prediction_horizons = [15, 30, 35]
slide_steps = [5, 10, 15]
test_series = test_series

for window_size in window_sizes:
    for prediction_horizon in prediction_horizons:
        for slide_step in slide_steps:
            ground_truths, input_windows = get_ground_truth(
                window_size, prediction_horizon, slide_step, test_series
            )

            predictions = expo_smoot.historical_forecasts(
                series=test_series,
                num_samples=1,
                train_length=window_size,
                start=None,
                forecast_horizon=prediction_horizon,
                stride=slide_step,
                retrain=True,
                overlap_end=False,
                last_points_only=False,
                verbose=False,
                show_warnings=True,
                predict_likelihood_parameters=False,
                enable_optimization=True,
            )

            meta_information_evaluation = {
                "Iterations": [],
                "MAE": [],
                "RMSE": [],
                "MAPE": [],
                "MSE": [],
                "input_window_size": [],
                "horizon": [],
                "stride": [],
            }

            stride = 0
            for i in range(len(predictions)):
                input_window = input_windows[i]
                ground_truth = ground_truths[i]
                forecast = predictions[i]
                sample = forecast.pd_dataframe().reset_index()
                sample["Close"] = df["Close"].apply(lambda val: 0 if val < 0 else val)
                sample = sample[["Date", "Close"]]
                forecast = TimeSeries.from_dataframe(
                    sample, time_col="Date", value_cols="Close", freq="1D"
                )

                bypass_information = {
                    "slide_step": stride,
                    "window_size": window_size,
                    "horizon": prediction_horizon,
                }
                make_plots(
                    input_window,
                    ground_truth,
                    forecast,
                    bypass_information,
                    result_plot_path,
                )

                actual = ground_truth.values().flatten().tolist()
                predicted = forecast.values().flatten().tolist()
                metrics = calculate_metrics(actual, predicted)

                meta_information_evaluation["Iterations"].append(stride)
                meta_information_evaluation["MAE"].append(metrics["MAE"])
                meta_information_evaluation["RMSE"].append(metrics["RMSE"])
                meta_information_evaluation["MAPE"].append(metrics["MAPE"])
                meta_information_evaluation["MSE"].append(metrics["MSE"])
                meta_information_evaluation["input_window_size"].append(window_size)
                meta_information_evaluation["horizon"].append(prediction_horizon)
                meta_information_evaluation["stride"].append(slide_step)

                stride += slide_step

            evaluation_df = pd.DataFrame.from_dict(meta_information_evaluation)
            evaluation_df.to_csv(
                f"{result_path}/window_size_{window_size}_horizon_{prediction_horizon}_stride_{slide_step}.csv",
                index=False,
            )
    #         break

    #     break
    # break

/tmp/ipykernel_9009/1889614651.py:12: RuntimeWarning: divide by zero encountered in divide
  mape = np.mean(np.abs((predicted - actual) / actual)) * 100
/tmp/ipykernel_9009/1889614651.py:12: RuntimeWarning: invalid value encountered in divide
  mape = np.mean(np.abs((predicted - actual) / actual)) * 100
/tmp/ipykernel_9009/1889614651.py:12: RuntimeWarning: divide by zero encountered in divide
  mape = np.mean(np.abs((predicted - actual) / actual)) * 100
/tmp/ipykernel_9009/1889614651.py:12: RuntimeWarning: divide by zero encountered in divide
  mape = np.mean(np.abs((predicted - actual) / actual)) * 100
/tmp/ipykernel_9009/1889614651.py:12: RuntimeWarning: invalid value encountered in divide
  mape = np.mean(np.abs((predicted - actual) / actual)) * 100
/tmp/ipykernel_9009/1889614651.py:12: RuntimeWarning: divide by zero encountered in divide
  mape = np.mean(np.abs((predicted - actual) / actual)) * 100
/tmp/ipykernel_9009/1889614651.py:12: RuntimeWarning: invalid value encountered in d

# Evaluate Metrics

In [ ]:
import pandas as pd
import glob


def aggregate_evaluation_results(file_pattern):
    eval_dict = {
        "window_size": [],
        "horizan": [],
        "stride": [],
        "AVG_MAE": [],
        "AVG_MSE": [],
        "AVG_RMSE": [],
        "AVG_MAPE": [],
    }

    paths = glob.glob(file_pattern)
    for path in paths:
        window_size = path.split("/")[-1].split("_")[2]
        horizan = path.split("/")[-1].split("_")[4]
        stride = path.split("/")[-1].split("_")[6].split(".")[0]

        df = pd.read_csv(path)
        eval_dict["window_size"].append(window_size)
        eval_dict["horizan"].append(horizan)
        eval_dict["stride"].append(stride)

        eval_dict["AVG_MAE"].append(df["MAE"].mean())
        eval_dict["AVG_MSE"].append(df["MSE"].mean())
        eval_dict["AVG_RMSE"].append(df["RMSE"].mean())
        eval_dict["AVG_MAPE"].append(df["MAPE"].mean())

    eval_df = pd.DataFrame.from_dict(eval_dict)
    eval_df = eval_df.dropna()
    eval_df.sort_values(
        ["window_size", "horizan", "stride"], inplace=True, ascending=True
    )

    return eval_df


# Example usage
file_pattern = f"../ProcessedData/Results/ExponentionalSmoothing/{fileName}/*.csv"
result_df = aggregate_evaluation_results(file_pattern)
result_df